In [10]:
!pip install google-generativeai pinecone-client sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [13]:
#!pip install -q langchain-google-genai
!pip install pinecone
import os
import google.generativeai as genai
from pinecone import Pinecone
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import files

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 16.5 MB/s eta 0:00:00


In [34]:
gemini_api_key = os.environ.get("GEMINI_API_KEY")
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
PINECONE_ENV = "us-east-1"
INDEX_NAME = "business-qa-bot"

In [35]:
genai.configure(api_key=GEMINI_API_KEY)

In [36]:
from pinecone import Pinecone

pc = Pinecone(api_key=PINECONE_API_KEY, environment=PINECONE_ENV)

In [16]:
uploaded = files.upload()
documents = []


Saving about_us.txt to about_us.txt
Saving product_info.txt to product_info.txt
Saving faq.txt to faq.txt
Saving policy.txt to policy.txt


In [17]:
for filename in uploaded.keys():
    with open(filename, 'r', encoding='utf-8') as f:
        text = f.read()
        documents.append({"text": text, "source": filename})


In [18]:
def split_text(text, chunk_size=500, overlap=50):
    chunks = []
    for i in range(0, len(text), chunk_size - overlap):
        chunk = text[i:i + chunk_size]
        if chunk:
            chunks.append(chunk)
    return chunks

In [19]:
texts = []
metadata = []

In [20]:
for doc in documents:
    chunks = split_text(doc['text'])
    texts.extend(chunks)
    metadata.extend([{"source": doc['source']}] * len(chunks))

In [21]:
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(texts).tolist()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [22]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY, environment=PINECONE_ENV)

if INDEX_NAME not in pc.list_indexes().names():
    pc.create_index(
        name=INDEX_NAME,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",            # or "gcp"
            region="us-east-1"      # match your Pinecone project's region
        )
    )

index = pc.Index(INDEX_NAME)




In [23]:
vectors = [(f"id-{i}", embedding, metadata[i]) for i, embedding in enumerate(embeddings)]
index.upsert(vectors=vectors)

{'upserted_count': 4}

In [24]:
#Step 8: Ask Questions with Gemini + Retrieved Context
def retrieve_context(query, top_k=3):
    query_embedding = model.encode([query]).tolist()[0]
    result = index.query(vector=query_embedding, top_k=top_k, include_metadata=True)
    return [match['metadata']['source'] + ":\n" + texts[int(match['id'].split('-')[-1])] for match in result['matches']]


In [37]:
from google.generativeai import GenerativeModel

client = GenerativeModel(GEMINI_API_KEY  )


In [27]:
from google.generativeai import GenerativeModel

model = GenerativeModel(model_name="models/gemini-2.5-pro")  # ✅ set model here

def ask_gemini(question):
    response = model.generate_content(
        contents=[
            {
                "role": "user",
                "parts": [{"text": question}]
            }
        ]
    )
    return response.candidates[0].content.parts[0].text

In [29]:
while True:
    user_input = input("Ask a question about your documents (or type 'exit'): ")
    if user_input.lower() == "exit":
        break
    answer = ask_gemini(user_input)
    print("\n📍 Answer:\n", answer)

Ask a question about your documents (or type 'exit'): what is your policy

📍 Answer:
 That's an excellent question. Since I'm an AI, "policy" can mean a few different things. I don't have policies in the way a company does, but I do operate based on a core set of principles and guidelines.

Here’s a breakdown of what that means:

### 1. My Core Operating Principles

Think of these as my internal rules for every interaction. My primary goal is to be **helpful and harmless**. This breaks down into several key areas:

*   **Safety First:** I am designed to decline requests for dangerous, unethical, illegal, or harmful content. This includes, but is not limited to:
    *   Hate speech, harassment, and discriminatory language.
    *   Instructions for illegal acts or dangerous activities (e.g., making weapons, self-harm).
    *   Sexually explicit or violent material.
    *   Spreading severe misinformation with the potential for real-world harm.
*   **Strive for Accuracy:** I aim to provid